# Gradio UI & LLM Integration

Note: Gradio uses user's current setting to choose the theme 'dark mode' or 'loght mode'.

# Add gradio in requirements.txt or pyproject.toml


In [7]:
## pyproject.toml

# [project]
# name = "llm-engineering"
# version = "0.1.0"
# requires-python = ">=3.11"
# dependencies = [
#     "gradio>=5.47.2",
#     # .. other dependencies
# ]

Import library

In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI

import gradio as gr

In [ ]:
load_dotenv()

TEST_API_KEY = os.getenv("TEST_API_KEY")
OLLAMA_NEW_BASE_URL = os.getenv("OLLAMA_BASE_URL")
GEMMA3_1B = os.getenv("GEMMA3_1B")

if (TEST_API_KEY or OLLAMA_NEW_BASE_URL or GEMMA3_1B) is None:
    raise ValueError("TEST_API_KEY or OLLAMA_NEW_BASE_URL or GEMMA3_1B environment variable is not set.")
else:
    print(f"Configurtion loaded successfully. \nTEST_API_KEY: {TEST_API_KEY[:10]}, \nOLLAMA_NEW_BASE_URL: {OLLAMA_NEW_BASE_URL}")

# build client
client = OpenAI(base_url=OLLAMA_NEW_BASE_URL, api_key=TEST_API_KEY)



Configurtion loaded successfully. 
TEST_API_KEY: DUMMY-API-, 
OLLAMA_NEW_BASE_URL: http://localhost:11434/v1


LLM API function to integrate with UI 

In [10]:
SYSTEM_PROMPT = """You are a helpful assistant that helps people find information.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
"""

# Initialize with user input
PAYLOAD = []
PAYLOAD.append({"role": "system", "content": SYSTEM_PROMPT})

In [17]:
# Take the request from user and call the LLM 
def chat(input: str) -> str:
    print(f"info: chat functions is called. \nInput: {input}")
    PAYLOAD.append({"role": "user", "content": input})
    response = client.chat.completions.create(model=GEMMA3_1B,
                                              messages=PAYLOAD, stream=False)
    msg = response.choices[0].message.content
    PAYLOAD.append({"role": "assistant", "content": msg})
    return msg

In [20]:
resp = chat("Hello, This is vivek from India. Tell me a funny fact.")

print(f"\nResponse: \n{resp}")

info: chat functions is called. 
Input: Hello, This is vivek from India. Tell me a funny fact.

Response: 
Okay, here’s a funny fact for you:

**Bananas are berries, but strawberries aren't.** 

It's a surprisingly complicated berry classification! 

---

Want to hear another one?


Create a Simple UI

In [ ]:

input_box = gr.Textbox(label="Ask me anything", placeholder="Type your question here...", type="text", lines=3, max_lines=10, max_length=100, autofocus=True)
output_box = gr.Textbox(label="Response", placeholder="You haven't asked anything yet...", type="text", lines=10, autoscroll=False, show_copy_button=True)

llm_chatting = gr.Interface(fn=chat, inputs=input_box, outputs=output_box, 
                    title="LLM Chat", 
                    description="ChatOllama", flagging_mode='never')

# Launch the Gradio interface
llm_chatting.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


info: chat functions is called. 
Input: Hi
info: chat functions is called. 
Input: what is the date today ?
info: chat functions is called. 
Input: who is the prime minister of india ?
info: chat functions is called. 
Input: what is the famous chess problem in DSA. the best approach to solve it ?
info: chat functions is called. 
Input: help me write a simple GUI in gradio to connect llm for chatting


Stop Server

In [22]:
llm_chatting.close()

# close all interfaces
gr.close_all()

Closing server running on port: 7860
